In [2]:
# load kategorisera dataset
import pandas as pd
import numpy as np

kategorisera_file = 'SE-en-bm003-Beds-category-with-items (5).xlsx'
kategorisera_df = pd.read_excel(kategorisera_file)

# drop missing values from Item_NO
kategorisera_df = kategorisera_df.dropna(subset=['Item_NO'])

#set Item_NO to int
kategorisera_df['Item_NO'] = kategorisera_df['Item_NO'].astype(int)

kategorisera_df['Item_NO'] = kategorisera_df['Item_NO'].apply(lambda x: str(x).zfill(8))

# text list of Item_NO
item_no_list = kategorisera_df['Item_NO'].tolist()

# list of strings separated by comma and having ''
item_no_list = [f"{item_no}" for item_no in item_no_list]

print(item_no_list)

c:\Users\QUTRA9\Workspace\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


['00349849', '00360484', '00458876', '00492677', '00505528', '00559371', '00571245', '00571250', '09099105', '09157421', '09189078', '09241302', '09302913', '09324284', '09392225', '09392230', '09406798', '09407024', '09494889', '09494912', '09495006', '09495049', '09495054', '09496628', '09496666', '09502691', '09514788', '09542687', '09548854', '09551436', '09551813', '09552657', '09553082', '09553162', '09553572', '09553727', '09553770', '09553794', '09553812', '09553826', '09555081', '09555118', '09555236', '09555321', '09555340', '09556127', '09559965', '09561954', '09569224', '09577771', '09577785', '09577790', '09577808', '09577813', '09577832', '09577846', '09577851', '09577865', '09577889', '09577894', '09577907', '09577926', '09577931', '09577945', '09577969', '09577988', '09577993', '09578006', '09578011', '09578049', '09578068', '09578092', '09578148', '09578172', '09578191', '09578209', '09578394', '09578450', '09902934', '09929373', '10453086', '10492672', '10538929', '19

In [3]:
import pandas as pd
import numpy as np
# Load the imported dataset
pkl_file = 'bed_merged_data.pkl'
df = pd.read_pickle(pkl_file)
df.shape

#drop columns with 100% missing values
df = df.dropna(axis=1, how='all')
print(df.shape)
df.info()

target_df = df[['Item_NO', 'Item_Type', 'Item_Name', 'Product_Name', 'Child1_Category_Name', 'features', 'length', 'width', 'Product_Type_Name', 'colour','material','HFB_Name', 'PA_Name','VALDES_TEXT_EN','PROD_TYPE_NAME_EN','Sales_start_date', 'End_date_of_sale']]
#drop duplicate
target_df = target_df.drop_duplicates(subset=['Item_NO'])
target_df.shape

#Check for missing values
missing_values = target_df.isnull().sum()
print(missing_values)



(4086, 133)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4086 entries, 0 to 4085
Columns: 133 entries, Selected_Category_ID to bk_shanawas
dtypes: Int64(2), float64(7), object(124)
memory usage: 4.2+ MB
Item_NO                    0
Item_Type                  0
Item_Name                  0
Product_Name               0
Child1_Category_Name       0
features                2070
length                  1926
width                   1614
Product_Type_Name          0
colour                   849
material                1033
HFB_Name                   0
PA_Name                    0
VALDES_TEXT_EN           981
PROD_TYPE_NAME_EN        976
Sales_start_date           0
End_date_of_sale        2440
dtype: int64


### To pull valid design data from bigquery eRIX item_valid_design_local_updated

In [4]:
#convert item_no to list to insert in bigquery
item_no = target_df['Item_NO'].tolist()
item_no = [str(i) for i in item_no]
item_no = ','.join(item_no)
print(item_no)

# with " " in the string
item_no = target_df['Item_NO'].tolist()
item_no = ['"'+str(i)+'"' for i in item_no]
item_no = ','.join(item_no)
print(item_no)

00349849,00360484,00458876,00492677,00505528,00559371,00571245,00571250,09099105,09157421,09189078,09241302,09302913,09324284,09392225,09392230,09406798,09407024,09494889,09494912,09495006,09495049,09495054,09496628,09496666,09502691,09514788,09542687,09548854,09551436,09551813,09552657,09553082,09553162,09553572,09553727,09553770,09553794,09553812,09553826,09555081,09555118,09555236,09555321,09555340,09556127,09559965,09561954,09569224,09577771,09577785,09577790,09577808,09577813,09577832,09577846,09577851,09577865,09577889,09577894,09577907,09577926,09577931,09577945,09577969,09577988,09577993,09578006,09578011,09578049,09578068,09578092,09578148,09578172,09578191,09578209,09578394,09578450,09902934,09929373,10453086,10492672,10538929,19019085,19019226,19022549,19099157,19175957,19175976,19188827,19241293,19408669,19441739,19494879,19494884,19494898,19494997,19495015,19496487,19496500,19496538,19496699,19501733,19502695,19542677,19542682,19548858,19548863,19551327,19551332,19552609,1

In [5]:
#read json file
import json

bed_json = 'bed_valdes_items_update.json' # 'bed_valdes_items.json'
with open(bed_json) as f:
    data = json.load(f)
    
#convert json to dataframe
bed_df = pd.DataFrame(data)
bed_df.shape
#change bed_df item_no to Item_NO
bed_df.rename(columns={'item_no':'Item_NO'}, inplace=True)

#join the two dataframes on Item_NO
target_merged_df = pd.merge(target_df, bed_df, on='Item_NO', how='left')

# for all missing values in valdes_text_en fill with valid_design_text
target_merged_df['VALDES_TEXT_EN'] = target_merged_df['VALDES_TEXT_EN'].fillna(target_merged_df['valid_design_text'])

target_all_df = target_merged_df.drop(columns=['valid_design_text','item_type']).copy()

#print item nos that do not have valdes_text_en from target_all_df
missing_valdes_text_en = target_all_df[target_all_df['VALDES_TEXT_EN'].isnull()]
print(missing_valdes_text_en['Item_NO'])

#Note: manually looking at the missing value in eRIX Prod database, turns out they are not in there.
#hints of ghost products?

#drop missing valdes_text_en
target_all_df = target_all_df.dropna(subset=['VALDES_TEXT_EN'])

2221    10032294
2240    00102065
2241    20535888
2242    30513541
2245    60102067
2246    80102066
Name: Item_NO, dtype: object


In [6]:
#look at summary statistics, numerical features
summary = target_all_df.describe()

summary.head()

,Item_NO,Item_Type,Item_Name,Product_Name,Child1_Category_Name,features,length,width,Product_Type_Name,colour,material,HFB_Name,PA_Name,VALDES_TEXT_EN,PROD_TYPE_NAME_EN,Sales_start_date,End_date_of_sale,country_code,language_code
count,3033,3033,3033,3033,3033,967,1111,1423,3033,2184,2006,3033,3033,3033,2058,3033,598,3031,3031
unique,3033,2,3029,103,9,11,33,113,155,13,10,4,13,512,83,108,12,1,1
top,00349849,SPR,LYNGÖR BJ20SS db 160x200 Vågstranda firm/lblue...,VIMLE,Sofa-beds,Washable cover,200 cm,160 cm,divan bed,grey,Fabric,Beds & Mattresses,Sofas,white,divan bed,2024-09-30T22:00:00Z,2025-06-30T22:00:00Z,SE,en
freq,1,2145,2,633,1392,396,300,200,548,780,1208,1621,809,64,548,878,205,3031,3031


In [7]:
import pandas as pd
import numpy as np
from datetime import datetime

# Convert the start_date and end_date to datetime
target_all_df['Sales_start_date'] = pd.to_datetime(target_all_df['Sales_start_date'])
target_all_df['End_date_of_sale'] = pd.to_datetime(target_all_df['End_date_of_sale'])

# Assuming 'today' is a datetime object
today = pd.Timestamp('now', tz='UTC')

# Filter the DataFrame with the converted 'today', include end date with missing values
active_products = target_all_df[(target_all_df['Sales_start_date'] <= today) & 
                                (target_all_df['End_date_of_sale'].isnull() | (target_all_df['End_date_of_sale'] >= today))
                                ]


# Sort active products by start date
sorted_active_products = active_products.sort_values('Sales_start_date')

# Display the sorted active products
print(sorted_active_products)

       Item_NO Item_Type                                          Item_Name  \
1444  60094595       ART                     POÄNG csh ach Glose dark brown   
1435  30094592       ART                    POÄNG csh ftst Glose dark brown   
2496  00102452       ART                       MYDAL bnk bd frm 90x200 pine   
1437  30105901       ART                       POÄNG csh ach Glose eggshell   
1436  30105897       ART                      POÄNG csh ftst Glose eggshell   
...        ...       ...                                                ...   
2165  20572197       ART  EKHOLMA cvr f crnr sf,4st w opn end Kilanda lb...   
2162  10572145       ART              EKHOLMA cvr f so2 Kilanda light beige   
664   79564336       SPR  TÄRNKULLEN uph bed frm 180x200 Tibb bei/grey L...   
1028  79517057       SPR  FRIHETEN csb+stor w ex bac cshns/Faringe brown...   
1106  80587704       ART                     VÅRKUMLA chair-bed Vansta blue   

     Product_Name Child1_Category_Name        featu

In [8]:
sorted_active_products.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2796 entries, 1444 to 1106
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   Item_NO               2796 non-null   object             
 1   Item_Type             2796 non-null   object             
 2   Item_Name             2796 non-null   object             
 3   Product_Name          2796 non-null   object             
 4   Child1_Category_Name  2796 non-null   object             
 5   features              733 non-null    object             
 6   length                878 non-null    object             
 7   width                 1189 non-null   object             
 8   Product_Type_Name     2796 non-null   object             
 9   colour                1948 non-null   object             
 10  material              1770 non-null   object             
 11  HFB_Name              2796 non-null   object             
 12  PA_Name 

In [9]:
def summarize_missing_columns(df):
    def get_missing_columns(row):
        missing_columns = [col for col in df.columns if pd.isnull(row[col])]
        return ', '.join(missing_columns) if missing_columns else 'None'
    
    df['missing_flags'] = df.apply(get_missing_columns, axis=1)
    return df

df_with_flags = summarize_missing_columns(active_products.drop(columns=['Sales_start_date', 'End_date_of_sale']))
df_with_flags.head()


,Item_NO,Item_Type,Item_Name,Product_Name,Child1_Category_Name,features,length,width,Product_Type_Name,colour,material,HFB_Name,PA_Name,VALDES_TEXT_EN,PROD_TYPE_NAME_EN,country_code,language_code,missing_flags
0,00349849,ART,NORDLI bed frm w storage 160x200 cm white,NORDLI,Double beds,Storage,202 cm,160 cm,bed frame with storage,white,Wood (including board),Beds & Mattresses,Bed frames and beds with non upholstered headb...,white,bed frame with storage,SE,en,None
1,00360484,ART,UTÅKER stckbl bed 80x200 pine 2-p,UTÅKER,Double beds,Stackable,205 cm,83 cm,stackable bed,beige,Solid wood,Beds & Mattresses,Flexible sleep solutions,pine,stackable bed,SE,en,None
2,00458876,ART,IDANÄS bed frm w storage 180x200 white,IDANÄS,Double beds,Storage,207 cm,191 cm,bed frame with storage,white,Wood (including board),Beds & Mattresses,Bed frames and beds with non upholstered headb...,white,bed frame with storage,SE,en,None
3,00492677,ART,KLEPPSTAD bed frm 140x200 white/Vissle beige,KLEPPSTAD,Double beds,Washable cover,207 cm,146 cm,bed frame,white,Metal,Beds & Mattresses,Bed with upholstered headboard,white/Vissle beige,bed frame,SE,en,None
4,00505528,ART,VEVELSTAD bed frm 140x200 white,VEVELSTAD,Double beds,None,207 cm,146 cm,bed frame,white,Metal,Beds & Mattresses,Bed frames and beds with non upholstered headb...,white,bed frame,SE,en,features


In [10]:
# drop item_no, item_type, missing_flags
df_with_flags_ml = df_with_flags.drop(columns=['Item_Type', 'missing_flags', 'colour', 'Child1_Category_Name', 'features', 'PROD_TYPE_NAME_EN'])
df_with_flags_ml.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2796 entries, 0 to 3038
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Item_NO            2796 non-null   object
 1   Item_Name          2796 non-null   object
 2   Product_Name       2796 non-null   object
 3   length             878 non-null    object
 4   width              1189 non-null   object
 5   Product_Type_Name  2796 non-null   object
 6   material           1770 non-null   object
 7   HFB_Name           2796 non-null   object
 8   PA_Name            2796 non-null   object
 9   VALDES_TEXT_EN     2796 non-null   object
 10  country_code       2795 non-null   object
 11  language_code      2795 non-null   object
dtypes: object(12)
memory usage: 284.0+ KB


## Cleaning missing

In [11]:
import re

# Use Item_Name and separate the 3digitx3digit pattern or 2digitx2digit pattern or 3digitx2digit pattern or 2digitx3digit pattern to length and width columns, only for missing values in length and width columns
def extract_length_width(row):
    if pd.isnull(row['length']) and pd.isnull(row['width']):
        item_name = row['Item_Name']
        match = re.search(r'(\d{2,3})x(\d{2,3})', item_name)
        if match:
            row['length'] = match.group(1)
            row['width'] = match.group(2)
    return row


# Function to fill missing values based on the most frequent value within the same product name series
def fill_missing_values(df):
    for column in df.columns:
        df[column] = df.groupby('Product_Name')[column].transform(lambda x: x.ffill().bfill())
    return df

# Function to fill missing values based on the same Product_Type_Name series
def fill_missing_values_Product_Type_name(df):
    for column in df.columns:
        df[column] = df.groupby('Product_Type_Name')[column].transform(lambda x: x.ffill().bfill())
        
    return df

# Function to fill missing values based on the same PA_Name series
def fill_missing_values_PA_name(df):
    for column in df.columns:
        df[column] = df.groupby('PA_Name')[column].transform(lambda x: x.ffill().bfill())
        
    return df

# Function to fill missing values based on the same HFB_Name series
def fill_missing_values_HFB_name(df):
    for column in df.columns:
        df[column] = df.groupby('HFB_Name')[column].transform(lambda x: x.ffill().bfill())
        
    return df

# Pipeline to fill missing values
def fill_missing_values_pipeline(df):
    df = fill_missing_values(df)
    df = fill_missing_values_Product_Type_name(df)
    df = fill_missing_values_PA_name(df)
    df = fill_missing_values_HFB_name(df)
    return df

#empty the length and width columns first before applying the function
df_with_flags_ml['length'] = np.nan
df_with_flags_ml['width'] = np.nan

df_filled= df_with_flags_ml.apply(extract_length_width, axis=1)
df_filled.info()

df_filled = fill_missing_values_pipeline(df_filled)

df_filled.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2796 entries, 0 to 3038
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Item_NO            2796 non-null   object
 1   Item_Name          2796 non-null   object
 2   Product_Name       2796 non-null   object
 3   length             1441 non-null   object
 4   width              1441 non-null   object
 5   Product_Type_Name  2796 non-null   object
 6   material           1770 non-null   object
 7   HFB_Name           2796 non-null   object
 8   PA_Name            2796 non-null   object
 9   VALDES_TEXT_EN     2796 non-null   object
 10  country_code       2795 non-null   object
 11  language_code      2795 non-null   object
dtypes: object(12)
memory usage: 284.0+ KB


C:\Users\QUTRA9\AppData\Local\Temp\ipykernel_26820\2102795283.py:17: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[column] = df.groupby('Product_Name')[column].transform(lambda x: x.ffill().bfill())
C:\Users\QUTRA9\AppData\Local\Temp\ipykernel_26820\2102795283.py:17: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[column] = df.groupby('Product_Name')[column].transform(lambda x: x.ffill().bfill())
C:\Users\QUTRA9\AppData\Local\Temp\ipykernel_26820\2102795283.py:17: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated

<class 'pandas.core.frame.DataFrame'>
Index: 2796 entries, 0 to 3038
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Item_NO            2796 non-null   object
 1   Item_Name          2796 non-null   object
 2   Product_Name       2796 non-null   object
 3   length             2781 non-null   object
 4   width              2781 non-null   object
 5   Product_Type_Name  2796 non-null   object
 6   material           2796 non-null   object
 7   HFB_Name           2796 non-null   object
 8   PA_Name            2796 non-null   object
 9   VALDES_TEXT_EN     2796 non-null   object
 10  country_code       2796 non-null   object
 11  language_code      2796 non-null   object
dtypes: object(12)
memory usage: 284.0+ KB


C:\Users\QUTRA9\AppData\Local\Temp\ipykernel_26820\2102795283.py:30: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[column] = df.groupby('PA_Name')[column].transform(lambda x: x.ffill().bfill())
C:\Users\QUTRA9\AppData\Local\Temp\ipykernel_26820\2102795283.py:30: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[column] = df.groupby('PA_Name')[column].transform(lambda x: x.ffill().bfill())
C:\Users\QUTRA9\AppData\Local\Temp\ipykernel_26820\2102795283.py:30: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will 

' 2   length             2923 non-null   object\n 3   width              3924 non-null   object\n 5   material           3482 non-null   object\n 8   VALDES_TEXT_EN     3026 non-null   object\n '

In [12]:
#check for missing values
missing_values = df_filled.isnull().sum()
print(missing_values)
df_filled.info()

Item_NO               0
Item_Name             0
Product_Name          0
length               15
width                15
Product_Type_Name     0
material              0
HFB_Name              0
PA_Name               0
VALDES_TEXT_EN        0
country_code          0
language_code         0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 2796 entries, 0 to 3038
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Item_NO            2796 non-null   object
 1   Item_Name          2796 non-null   object
 2   Product_Name       2796 non-null   object
 3   length             2781 non-null   object
 4   width              2781 non-null   object
 5   Product_Type_Name  2796 non-null   object
 6   material           2796 non-null   object
 7   HFB_Name           2796 non-null   object
 8   PA_Name            2796 non-null   object
 9   VALDES_TEXT_EN     2796 non-null   object
 10  country_code       2796 non-

In [13]:
#drop missing values in length
df_filled = df_filled.dropna(subset=['length'])
df_filled.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2781 entries, 0 to 3038
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Item_NO            2781 non-null   object
 1   Item_Name          2781 non-null   object
 2   Product_Name       2781 non-null   object
 3   length             2781 non-null   object
 4   width              2781 non-null   object
 5   Product_Type_Name  2781 non-null   object
 6   material           2781 non-null   object
 7   HFB_Name           2781 non-null   object
 8   PA_Name            2781 non-null   object
 9   VALDES_TEXT_EN     2781 non-null   object
 10  country_code       2781 non-null   object
 11  language_code      2781 non-null   object
dtypes: object(12)
memory usage: 282.4+ KB


Data complete

In [14]:
#data complete
df_clean = df_filled.copy()

#show duplicates in item_name
duplicates = df_clean[df_clean.duplicated(subset=['Item_Name'], keep=False)]
duplicates = duplicates.sort_values(by='Item_Name')
duplicates.head()

,Item_NO,Item_Name,Product_Name,length,width,Product_Type_Name,material,HFB_Name,PA_Name,VALDES_TEXT_EN,country_code,language_code


In [15]:
#convert to categorical for all columns except length and width which should be int64
for column in df_clean.columns:
    if column not in ['length', 'width']:
        df_clean[column] = df_clean[column].astype('category')
    else:
        df_clean[column] = df_clean[column].astype('int64')
        

#df_filled_ml_copy = df_filled_ml.drop(columns=['Item_Name','HFB_Name']).copy()
df_filled_ml_copy = df_clean.drop(columns=['Item_Name','HFB_Name']).copy()
df_filled_ml_original = df_filled_ml_copy.copy()

#df_filled_ml_copy = df_filled_ml_copy.drop(columns=['Item_NO']).drop_duplicates()
df_filled_ml_copy.info()

# save to csv
df_filled_ml_copy.to_csv('bed_cleaned_data_2025_v2.csv')


<class 'pandas.core.frame.DataFrame'>
Index: 2781 entries, 0 to 3038
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   Item_NO            2781 non-null   category
 1   Product_Name       2781 non-null   category
 2   length             2781 non-null   int64   
 3   width              2781 non-null   int64   
 4   Product_Type_Name  2781 non-null   category
 5   material           2781 non-null   category
 6   PA_Name            2781 non-null   category
 7   VALDES_TEXT_EN     2781 non-null   category
 8   country_code       2781 non-null   category
 9   language_code      2781 non-null   category
dtypes: category(8), int64(2)
memory usage: 210.4 KB
    Item_NO Product_Name  length  width       Product_Type_Name  \
0  00349849       NORDLI     160    200  bed frame with storage   
1  00360484       UTÅKER      80    200           stackable bed   
2  00458876       IDANÄS     180    200  bed frame with 